In [1]:
import os
import pandas as pd

import ssl
import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

import ywc_email_template

In [3]:
campers = pd.read_csv("certs-and-slips/cert_list.csv")
campers.sort_values("id_with_nickname", inplace=True)
campers.reset_index(drop=True, inplace=True)
campers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id_with_nickname        80 non-null     object
 1   major                   80 non-null     object
 2   fullname_th_with_title  80 non-null     object
 3   fullname_en             80 non-null     object
 4   email                   80 non-null     object
 5   groub_name              80 non-null     object
 6   awards                  26 non-null     object
dtypes: object(7)
memory usage: 4.5+ KB


In [5]:
def extract_title(fullname_th_with_title):
    if fullname_th_with_title[:3] == "นาย":
        return "นาย"
    elif fullname_th_with_title[:6] == "นางสาว":
        return "นางสาว"
    elif fullname_th_with_title[:3] == "นาง":
        return "นาง"
    
def extract_name(fullname_th_with_title):
    if fullname_th_with_title[:3] == "นาย":
        return fullname_th_with_title[3:]
    elif fullname_th_with_title[:6] == "นางสาว":
        return fullname_th_with_title[6:]
    elif fullname_th_with_title[:3] == "นาง":
        return fullname_th_with_title[3:]

campers["nametitle_th"] = campers["fullname_th_with_title"].apply(extract_title)
campers["fullname_th"] = campers["fullname_th_with_title"].apply(extract_name)
campers["id"] = campers["id_with_nickname"].apply(lambda x: x[1:6])
campers["is_awarded"] = campers["awards"].notna()

In [7]:
import getpass

sender_email = "ywc@webmaster.or.th"
sender_password = getpass.getpass("Password: ")

In [11]:
context = ssl.create_default_context()

with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as smtp:
    smtp.login(sender_email, sender_password)
    for camper in list(campers.itertuples())[2:]:
        try:
            msg = MIMEMultipart()

            msg["From"] = sender_email
            msg["To"] = camper.email
            recipients = [camper.email]

            msg["Subject"] = (
                "[YWC19] จัดส่งเกียรติบัตรและหลักฐานการโอนเงินสำหรับผู้เข้าร่วมโครงการ - "
                + camper.fullname_th
            )

            msgText = MIMEText(
                ywc_email_template.build_cert_email(
                    camper_title="คุณ",
                    camper_fullname=camper.fullname_th,
                    is_awarded=camper.is_awarded,
                ),
                "html",
            )
            msg.attach(msgText)
            with open(
                "certs-and-slips/Participation Certs/"
                + camper.id_with_nickname
                + "_YWC19_Participation.png",
                "rb",
            ) as file:
                cert_participation_file = MIMEApplication(file.read())
                cert_participation_file.add_header(
                    "Content-Disposition",
                    "attachment",
                    filename=f"[YWC19] เกียรติบัตรรับรองการเข้าร่วมโครงการ - {camper.id}.png",
                )
            msg.attach(cert_participation_file)

            if camper.is_awarded:
                for award in camper.awards.split(", "):
                    award_filename = award.replace(" ", "_")
                    if award[:5] != "Jitta":
                        award_filename = "YWC19_" + award_filename
                    award_filename = camper.id_with_nickname + "_" + award_filename + ".png"
                    with open(
                        "certs-and-slips/Award Certs/" + award_filename, "rb"
                    ) as file:
                        cert_award_file = MIMEApplication(file.read())
                        cert_award_file.add_header(
                            "Content-Disposition",
                            "attachment",
                            filename=f"[YWC19] เกียรติบัตรรางวัล {award} - {camper.id}.png",
                        )
                    msg.attach(cert_award_file)

            with open(f"certs-and-slips/Deposit Slips/{camper.id}.JPG", "rb") as file:
                slip_file = MIMEApplication(file.read())
                slip_file.add_header(
                    "Content-Disposition",
                    "attachment",
                    filename=f"[YWC19] หลักฐานการโอนคืนเงินมัดจำ - {camper.id}.jpg",
                )
            msg.attach(slip_file)

            smtp.sendmail(
                sender_email,
                recipients,
                msg.as_string(),
            )
            print(f"Successfully sent {camper.id}")
        except Exception as err:
            print(f"Failed {camper.id}")
            print(err)


Successfully sent CT006
Successfully sent CT009
Successfully sent CT012
Successfully sent CT013
Successfully sent CT020
Successfully sent CT021
Successfully sent CT022
Successfully sent CT026
Successfully sent CT027
Successfully sent CT029
Successfully sent CT030
Successfully sent CT031
Successfully sent CT032
Successfully sent CT033
Successfully sent CT034
Successfully sent CT037
Successfully sent CT038
Successfully sent CT042
Successfully sent DS004
Successfully sent DS007
Successfully sent DS009
Successfully sent DS021
Successfully sent DS022
Successfully sent DS025
Successfully sent DS031
Successfully sent DS034
Successfully sent DS038
Successfully sent DS041
Successfully sent DS046
Successfully sent DS048
Successfully sent DS050
Successfully sent DS056
Successfully sent DS063
Successfully sent DS070
Successfully sent DS072
Successfully sent DS073
Successfully sent DS074
Successfully sent DS083
Successfully sent MK001
Successfully sent MK002
Successfully sent MK003
Successfully sen